In [7]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import re
import requests
from bs4 import BeautifulSoup

import time
from pathlib import Path

# GetDate
from datetime import datetime #,  date

In [8]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = './ARQ/ListagemExport_checkpoint_10-12-2022_10-03-30_2.csv'

listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

In [11]:
display(listagem)

,Projeto,Ano,Tipo,Ementa,Situacao,Autor,Codigo_Tipo,Id_Projeto_Camara,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,1,2017,Pedido de Informação FiscalizaJF,Solicita informações referentes a praça Adalbe...,Arquivada,"Dr. Antônio Aguiar, Rodrigo Mattos, Delegada S...",PEDFF,59295,14/02/2017,SIM,NAO
1,1,2017,Pedido de Informação,Solicita informações sobre as Clínicas Terapêu...,Arquivada,Cido Reis,PEDIF,58361,04/01/2017,SIM,SIM


In [10]:
# ADICIONAR CONTROLE DE EXECUÇOES, FALHA E EXPORTAÇAO AUTOMATICA
# Preenchimento do campo Id_Projeto_Camara
count = 1
failed_req= 0
failed = False
id_projeto_camara = ''
max_failed_req = 0
notfound_count = 0
req_count = 0
max_req = 150
export = False

fnd_dt = ''
data_projeto = ''
data = ''
fnd_tramitacao = ''
tramitacao = ''
fim_tramitacao = ''

fnd_tramitacao_f = ''
fnd_tramitacao_e = ''
fnd_tramitacao_compl = []
ultimo = ''
aprovado = ''
notfound_count_dt = 0
notfound_count_tram = 0
notfound_count_aprv = 0


while count > 0:
    for i in listagem.index:
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto_Camara'][i] != '': 
            with requests.Session() as session:
                proposicao = False

                id_proj = listagem['Id_Projeto_Camara'][i]
                
                # Variaveis de resposta
                #status = ''
                
                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                    resp_format = BeautifulSoup(resposta.content, "html.parser")

                    req_count+=1
                    
                    try:
                        # Identificacao da data dentro do retorno HTML
                        fnd_dt = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                          ).find('table', {'width':'98%', 'border':'0'}
                                                                ).find('td',{'bgcolor':'#FFFFFF'}
                                                                          ).find_all('font',{'size':'2'})
                        data = str(fnd_dt[0]).replace(' ', '').upper()
                        data_projeto = re.search('PERÍODO:(.+?)A', data).group(1)

                    except:
                        data_projeto = '31/12/2000'
                        notfound_count_dt += 1

                    try:
                         # Descrição explícita do fim da tramitação dentro do retorno HTML
                        fnd_tramitacao = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                          ).find('table', {'width':'98%', 'border':'0'}
                                                                ).find_all('p',{'align':'justify'})#.find('font',
                                                             #                           {'size':'2'})
                        tramitacao = str(fnd_tramitacao[1]).replace('	', '').replace(' ', '').upper()


                        if(re.search('-FIMTRAMITAÇÃO:SIM', tramitacao)) or (re.search('TRANSFORMADOEMNORMA', tramitacao)): 

                            fim_tramitacao = 'SIM'
                        else:

                            fim_tramitacao = 'NAO' 
                    
                    except:
                        fim_tramitacao = 'Informacao nao encontrada' 
                        notfound_count_tram += 1 
                    
                    
                    
    
                    # Identificacao das informacoes da proposicao e do retorno sobre as atividades de tramitacao, buscando pela identificaçao da aprovacao no HTML
                    #if 'Aprovad' in 'Situacao' or 'Trasformado em Norma Jur' in 'Situacao':
                    if 'Aprovad' in listagem['Situacao'][i] or 'Transformado em Norma' in listagem['Situacao'][i] or 'Transformada em Norma' in listagem['Situacao'][i]:
                        aprovado = 'SIM'
                    elif (listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF') and fim_tramitacao == 'NAO') or (listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF') and listagem['Situacao'][i] == 'Arquivada') or ('Retirad' in listagem['Situacao'][i]) or ('Rejeitad' in listagem['Situacao'][i]) or ('Vetad' in listagem['Situacao'][i]):# era 'Vetado Totalmente'\
                    #elif ('Codigo_Tipo' not in ('REQ', 'MOC', 'REP', 'PEDIF') and 'Situacao' == 'Arquivada') or ('Retirad' in 'Situacao') or ('Rejeitad' in 'Situacao') or ('Vetad' in 'Situacao') or ('Codigo_Tipo' not in ('REQ', 'MOC', 'REP', 'PEDIF') and fim_tramitacao == 'NAO'):# era 'Vetado Totalmente'\
                        aprovado = 'NAO'
                    else:
                        try:
                            # Limpeza para pegar o id do projeto dentro da prefeitura
                            fnd_tramitacao_f = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                          ).find('table', {'width':'98%', 'border':'0'}
                                                                ).find_all('td',{'bgcolor':'#FFFFFF'}
                                                                          )
                            fnd_tramitacao_e = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                                  ).find('table', {'width':'98%', 'border':'0'}
                                                                        ).find_all('td',{'bgcolor':'#EEEEEE'}
                                                                                  )

                            maxlen = max(len(fnd_tramitacao_f),len(fnd_tramitacao_e))
                            c = 0 

                            while c < maxlen:
                                if c < len(fnd_tramitacao_f):
                                    fnd_tramitacao_compl.append(fnd_tramitacao_f[c])

                                if c < len(fnd_tramitacao_e):
                                    fnd_tramitacao_compl.append(fnd_tramitacao_e[c])

                                c+=1

                            ultimo = str(fnd_tramitacao_compl[len(fnd_tramitacao_compl)-1]).replace('	', '').replace(' ', '').upper()

                            if(re.search('APROVAD', ultimo)):
                                aprovado = 'SIM'
                            else:
                                aprovado = 'NAO'
                                
                        except:
                            aprovado = 'Informacao nao encontrada'
                            notfound_count_aprv += 1

                    
                    
                    
                except:
                    #Assinala que houve falha
                    failed = True
                    time.sleep(180)
                    #print(resposta.status_code)
                    
                
                       
            session.close()
            time.sleep(0.5)
            
            listagem['Aprovado'][i] = aprovado
            listagem['Tramitacao_Encerrada'][i] = fim_tramitacao
            listagem['Data_Projeto'][i] = data_projeto
            
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve X falhas consecutivas ou houve mais de Y requisicoes consecutivas 
        if failed_req > max_failed_req or req_count > max_req:
            break

    if failed_req > max_failed_req or req_count > max_req:
        break  
  
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1


now = datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

#try:
if True:
# Exportacao da listagem no estado atual para arquivo csv
    file = 'ListagemExport_checkpoint_' + dthr_finalizado
    #path_to_file = '../ListagemExp/'+file+'.csv'
    path_to_file = './'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

else:
    export = False



if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
#print(situacao)
#print(tipo) 
#print(ementa)
#print(id_projeto_camara)


Fim do processo. Nao encontrados: data = 0, tramitacao = 0, aprovador = 0. Finalizado em: 10-12-2022_11-03-18. Status da exportacao: True.
